In [1]:
from matplotlib import pyplot
import plotly.io as pio
pio.renderers.default = "iframe"



In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import logging

# Minimal version for reliable deployment
logging.basicConfig(level=logging.INFO)

In [1]:
import simple_dashboard
simple_dashboard.get_data_and_create_simple_dashboard()

YF.download() has changed argument auto_adjust default to True


INFO:root:Simple dashboard created successfully!
INFO:root:Status: ✅ NORMAL | Price: $6173.07 | σ: 1.52


True

In [3]:
# Get S&P 500 data
end_date = datetime.now()
start_date = end_date - timedelta(days=300)

sp500 = yf.download('^GSPC', start=start_date, end=end_date, progress=False)
closes = sp500['Close'].dropna()

# Calculate metrics
ma_200 = closes.rolling(window=200).mean()
std_200 = closes.rolling(window=200).std()

current_price = closes.iloc[-1].values[0]
current_ma = ma_200.iloc[-1].values[0]
current_std = std_200.iloc[-1].values[0]
std_away = (current_price - current_ma) / current_std

# Get last 30 days for chart data
last_30_days = closes.tail(30)
ma_30_days = ma_200.tail(30)

# Convert to lists for JavaScript
dates = [d.strftime('%Y-%m-%d') for d in last_30_days.index]
# prices = last_30_days.tolist()
prices = last_30_days["^GSPC"].values.tolist()
# mas = ma_30_days.tolist()
mas = ma_30_days["^GSPC"].values.tolist()

# Determine status
abs_std = abs(std_away)
if 2.0 <= abs_std <= 3.0:
    status = "🚨 ALERT"
    status_color = "#ff4444"
elif abs_std < 2.0:
    status = "✅ NORMAL"
    status_color = "#44ff44"
else:
    status = "⚠️ EXTREME"
    status_color = "#ff8800"

direction = "above" if std_away > 0 else "below"

# Create HTML with embedded Chart.js
html_content = f"""

    color: {status_color};
    <p>Last Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S ET')}</p>
        <div class="metric-value">${current_price:.2f}</div>
        <div class="metric-value">${current_ma:.2f}</div>
        <div class="metric-value">{direction.title()}</div>
    <p><strong>Current Status:</strong> The S&P 500 is <strong>{abs_std:.2f} standard deviations {direction}</strong> its 200-day moving average.</p>
        labels: {dates},
            data: {prices},
            data: {mas},
"""



# Save JSON data
data_json = {
    'timestamp': datetime.now().isoformat(),
    'current_price': float(current_price),
    'ma_200': float(current_ma),
    'std_200': float(current_std),
    'std_away': float(std_away),
    'status': status,
    'direction': direction,
    'alert': 2.0 <= abs_std <= 3.0
}


# logging.info(f"Status: {status} | Price: ${current_price:.2f} | σ: {std_away:.2f}")

In [6]:
2.0 <= abs_std <= 3.0

False